In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [3]:
metadata = pd.read_csv("C:\Maya\CS\AnomalyDetectionMicrobiome\Human_Microbiome_Project_1_6_2017_metadata.tab",sep='\t')

In [4]:
metadata.head()

,#SampleID,BarcodeSequence,LinkerPrimerSequence,run_prefix,body_habitat,body_product,body_site,bodysite,dna_extracted,elevation,...,sample_type,scientific_name,sequencecenter,taxon_id,title,qiita_study_title,qiita_study_alias,qiita_owner,qiita_principal_investigator,Description
0,1927.SRS055168.SRX020678.SRR048038,TCAGACTCCTC,ATTACCGCGGCTGCTGG,SRR048038,UBERON:skin,UBERON:sebum,UBERON:skin of elbow,R_Antecubital_fossa,True,97.0,...,Skin,human skin metagenome,BCM,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS055168.SRX020678.SRR048038
1,1927.SRS015346.SRX020577.SRR044088,TCAGAACCAGC,ATTACCGCGGCTGCTGG,SRR044088,UBERON:vagina,UBERON:mucus,UBERON:central vagina,Mid_vagina,True,97.0,...,Vaginal,human vaginal metagenome,WUGSC,1632839,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS015346.SRX020577.SRR044088
2,1927.SRS021613.SRX020675.SRR047757,TCAGAGCTTC,ATTACCGCGGCTGCTGG,SRR047757,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,Tongue_dorsum,True,97.0,...,Oral,human oral metagenome,BCM,447426,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS021613.SRX020675.SRR047757
3,1927.SRS022512.SRX020678.SRR048023,TCAGTATCAAC,ATTACCGCGGCTGCTGG,SRR048023,UBERON:skin,UBERON:sebum,UBERON:skin of elbow,L_Antecubital_fossa,True,97.0,...,Skin,human skin metagenome,BCM,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS022512.SRX020678.SRR048023
4,1927.SRS016305.SRX020525.SRR044526,TCAGCCACGGTC,ATTACCGCGGCTGCTGG,SRR044526,UBERON:skin,UBERON:sebum,UBERON:skin of external ear,L_Retroauricular_crease,True,97.0,...,Skin,human skin metagenome,WUGSC,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS016305.SRX020525.SRR044526


In [5]:
data = pd.read_csv("C:\Maya\CS\AnomalyDetectionMicrobiome\Human_Microbiome_Project_1_6_2017_data.tab",sep='\t')

In [6]:
data.head()

,#OTU ID,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,...,1928.SRS042698.SRX020530.SRR045847,1928.SRS023827.SRX021470.SRR052699,1928.SRS022769.SRX022226.SRR058089,1928.SRS024716.SRX021470.SRR052699,1928.SRS045191.SRX021470.SRR052699,1928.SRS045171.SRX021470.SRR052699,1928.SRS024704.SRX021470.SRR052699,1928.SRS020726.SRX022239.SRR058109,1928.SRS022067.SRX022231.SRR058120,1928.SRS023274.SRX022223.SRR058086
0,4376890,6.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,188753,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,114076,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,253429,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4467992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### extract relevant samples

In [7]:
filtered_data_with_OTUs = data.loc[:, data.sum() > 3000] #5728 samples, 13268 features
filtered_data = filtered_data_with_OTUs.iloc[:, 1:]
filtered_data

,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,1927.SRS014913.SRX020546.SRR043697,...,1928.SRS042246.SRX020686.SRR048627,1928.SRS057075.SRX022222.SRR058085,1928.SRS043338.SRX022227.SRR058090,1928.SRS022777.SRX022226.SRR058089,1928.SRS016730.SRX019682.SRR040947,1928.SRS021283.SRX022235.SRR058095,1928.SRS018288.SRX019687.SRR041227,1928.SRS016559.SRX019636.SRR040605,1928.SRS016559.SRX019635.SRR040662,1928.SRS018292.SRX019687.SRR041262
0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### relative abundance - calc sum

In [8]:
final_data = filtered_data.copy()

for colname in filtered_data.columns:
    final_data.loc[:,colname] = filtered_data[colname] / sum(filtered_data[colname])

final_data.head()

,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,1927.SRS014913.SRX020546.SRR043697,...,1928.SRS042246.SRX020686.SRR048627,1928.SRS057075.SRX022222.SRR058085,1928.SRS043338.SRX022227.SRR058090,1928.SRS022777.SRX022226.SRR058089,1928.SRS016730.SRX019682.SRR040947,1928.SRS021283.SRX022235.SRR058095,1928.SRS018288.SRX019687.SRR041227,1928.SRS016559.SRX019636.SRR040605,1928.SRS016559.SRX019635.SRR040662,1928.SRS018292.SRX019687.SRR041262
0,0.001055,0.001894,0.000218,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000047,0.000213,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000248,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001515,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000164,0.00024,0.000265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Isolation Forest
data for IF

In [9]:
all_feces = metadata[metadata['body_site'] == 'UBERON:feces']['#SampleID']
feces = [sample for sample in all_feces if sample in final_data.columns]
feces_data = final_data[feces] #418 samples

#all _body sites except of feces
all_body_sites = metadata[metadata['body_site'] != 'UBERON:feces' ]['body_site'].drop_duplicates()
print(all_body_sites)

0                    UBERON:skin of elbow
1                   UBERON:central vagina
2                           UBERON:tongue
4             UBERON:skin of external ear
5                            UBERON:mouth
6                            UBERON:tooth
8                  UBERON:palatine tonsil
11                   UBERON:buccal mucosa
14               UBERON:vaginal introitus
17                          UBERON:throat
18                         UBERON:gingiva
19                  UBERON:anterior nares
20                     UBERON:hard palate
64      UBERON:posterior fornix of vagina
3534                  UBERON:skin of nose
3537                        UBERON:saliva
Name: body_site, dtype: object


In [ ]:
all_auc = []

for body_site in all_body_sites:
    now_auc = []
    i = 0
    while i < 100:
        all_body_site = metadata[metadata['body_site'] == body_site]['#SampleID']
        final_all_body_site = [sample for sample in all_body_site if sample in  final_data.columns]
        all_body_site_data = final_data[final_all_body_site]
        few_all_body_site_data = all_body_site_data.iloc[: ,-42:] #42 samples
        #################choose random 42 ###############
        
        inputIF = pd.concat([feces_data, few_all_body_site_data], axis=1) #42 samples out of 460 should be outliers

        model = IsolationForest(n_estimators=100)
        model.fit(inputIF.transpose())  # fit 100 trees

        scores = model.decision_function(inputIF.transpose()) #higher score means more normal
        #sortedScores = sorted(scores)

        # indices of anomalies (body site != feces) in sorted scores. higher score = higher indices = more normal !
        #anomalies_indices_in_sorted = [i for i in range(len(sortedScores)) if sortedScores[i] in scores[-42:]]
        #print(anomalies_indices_in_sorted) # if algorithm is working ok, indices are expected to be low. in reality we get differernt indices, some are very high. 

        results = [1 for i in range(418)]
        more_res = [0 for i in range(42)]
        y = results + more_res

        # calculate area under the curve
        now_auc.append(roc_auc_score(y, scores))
        i += 1
        
    all_auc.append(now_auc)

        
print(all_auc)

In [29]:
all_feces = metadata[metadata['body_site'] == 'UBERON:feces']['#SampleID']
all_skin_of_nose = metadata[metadata['body_site'] == 'UBERON:skin of nose']['#SampleID']

feces = [sample for sample in all_feces if sample in final_data.columns]
skin_of_nose = [sample for sample in all_skin_of_nose if sample in  final_data.columns]

feces_data = final_data[feces] #418 samples

skin_of_nose_data = final_data[skin_of_nose] #203 samples
few_skin_of_nose_data = skin_of_nose_data.iloc[: ,-42:] #42 samples

inputIF = pd.concat([feces_data, few_skin_of_nose_data], axis=1) #42 samples out of 460 should be outliers

### IF model

In [17]:
'''loop 100 times for each part (5 parts) of body, BoxFlot / KDE plot, AUC an a function of body site'''

model = IsolationForest(n_estimators=100)
model.fit(inputIF.transpose())  # fit 100 trees
 
scores = model.decision_function(inputIF.transpose()) #higher score means more normal
sortedScores = sorted(scores)

# indices of anomalies (skin of nose) in sorted scores. higher score = higher indices = more normal !
anomalies_indices_in_sorted = [i for i in range(len(sortedScores)) if sortedScores[i] in scores[-42:]]
print(anomalies_indices_in_sorted) # if algorithm is working ok, indices are expected to be low. in reality we get differernt indices, some are very high. 

results = [1 for i in range(418)]
more_res = [0 for i in range(42)]
y = results + more_res

# calculate area under the curve
roc_auc_score(y, scores) #around 0.35


[38, 51, 76, 130, 136, 148, 181, 205, 216, 245, 253, 260, 261, 265, 268, 287, 298, 299, 301, 317, 319, 322, 337, 360, 364, 366, 372, 383, 384, 393, 395, 402, 411, 418, 426, 431, 436, 441, 446, 447, 450, 453, 454, 455, 456, 457, 458, 459]


0.30889724310776945

graph depth of outlier vs normal
mann whitney test SCIPY
different body sites
bootstrap